In [279]:
import numpy as np
import pandas as pd

import re
import datetime as dt

from sklearn.preprocessing import OneHotEncoder

# 0. Reading files
- dataset
- metadata of the dataset

In [280]:
df = pd.read_parquet('data/catB_train.parquet')
display(df)

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_giclaim,giclaim_cnt_success,recency_giclaim_success,giclaim_cnt_unsuccess,recency_giclaim_unsuccess,flg_gi_claim_29d435_ever,flg_gi_claim_058815_ever,flg_gi_claim_42e115_ever,flg_gi_claim_856320_ever,f_purchase_lh
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,3363260248,None,Singapore,P,LAPSED,2016-08-01,1994-05-24,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
11964,c3509f0aef,Chinese,Singapore,P,ACTIVE,1990-03-20,1948-12-16,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
5390,6e44cfb3e7,Chinese,Singapore,P,ACTIVE,2007-12-31,1967-12-06,Female,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN
860,dd4e362f9f,Others,Singapore,P,ACTIVE,2007-09-01,1986-04-01,Male,0.0,0.0,...,NaN,None,None,None,None,None,None,None,None,NaN


In [281]:
metadf = pd.read_csv("data/metadata.csv", encoding='cp1252')
display(metadf)

,column_name,is_numeric,is_date,to_delete,requires_preprocessing,remarks
0,affcon_visit_days,1,0,0,0.0,Metric related to client’s activity in affinit...
1,annual_income_est,0,0,0,1.0,This column is the client’s income bracket. Wi...
2,ape_507c37,1,0,0,0.0,NaN
3,ape_839f8a,1,0,0,0.0,NaN
4,ape_c4bda5,1,0,0,0.0,"This column is 0 for all, except 5 rows. But t..."
...,...,...,...,...,...,...
241,sumins_lh_f852af,1,0,0,NaN,NaN
242,sumins_ltc,1,0,0,NaN,NaN
243,sumins_ltc_43b9d5,1,0,0,NaN,NaN
244,tot_cancel_pols,1,0,0,NaN,NaN


# 1. Removing Unnecessary Columns
- Columns with constant values
- Columns with all NaN
- Columns that are aggregations of other columns:  
    1. ape_ltc = ape_grp_de05ae + ape_grp_6a5788 + ape_ltc_43b9d5

In [282]:
aggregated_cols = ["ape_ltc"]

In [283]:
n_unique_values_per_col = df.nunique(dropna=False).\
        reset_index().\
        rename(columns={"index" : "column", 0 : "n_unique_values"}).\
        sort_values("column").\
        sort_values("n_unique_values")

useless_cols_df = n_unique_values_per_col.\
        loc[n_unique_values_per_col.n_unique_values == 1].\
        sort_values("column").\
        reset_index(drop=True)

display(useless_cols_df)

,column,n_unique_values
0,ape_32c74c,1
1,ape_d0adeb,1
2,ape_gi,1
3,ape_gi_058815,1
4,ape_gi_29d435,1
5,ape_gi_42e115,1
6,ape_gi_856320,1
7,ape_gi_a10d1b,1
8,ape_inv_dcd836,1
9,ape_lh_d0adeb,1


In [284]:
useless_cols = list(useless_cols_df.column)
df = df.drop(columns=useless_cols + aggregated_cols).reset_index(drop=True)
display(df)

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,flg_hlthclaim_839f8a_ever,recency_hlthclaim_839f8a,flg_hlthclaim_14cb37_ever,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,f_purchase_lh
0,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
3,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17987,3363260248,None,Singapore,P,LAPSED,2016-08-01,1994-05-24,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
17988,c3509f0aef,Chinese,Singapore,P,ACTIVE,1990-03-20,1948-12-16,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
17989,6e44cfb3e7,Chinese,Singapore,P,ACTIVE,2007-12-31,1967-12-06,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
17990,dd4e362f9f,Others,Singapore,P,ACTIVE,2007-09-01,1986-04-01,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


# 2. Casting of Data Types

In [285]:
numerical_check = (metadf.is_numeric == 1) & (metadf.is_date == 0)
numerical_cols = metadf.loc[numerical_check].column_name.to_list()

date_check = (metadf.is_numeric == 1) & (metadf.is_date == 1)
date_cols = metadf.loc[date_check].column_name.to_list()

categorical_check = metadf.is_numeric == 0
categorical_cols = metadf.loc[categorical_check].column_name.to_list()

len(numerical_cols) + len(date_cols) + len(categorical_cols) == df.shape[1]

True

In [286]:
# further split categorical columns to 1. cat_cols: columns that do not require OHE, 2. cat_ohe_cols: columns that require OHE
categorical_regex = re.compile('^((f_)|(flg_)|(is_))')
cat_cols = [col for col in categorical_cols if categorical_regex.match(col)]
cat_ohe_cols = list(set(categorical_cols) - set(cat_cols)) # to create OHE under Section (4)
cat_ohe_cols.remove("clntnum") # remove PK

len(categorical_cols) == len(cat_cols) + len(cat_ohe_cols) + 1 # 1 for PK

True

In [287]:
df[numerical_cols] = df[numerical_cols].astype(float)
df[date_cols] = df[date_cols].apply(lambda x: pd.to_datetime(x, errors='coerce', format='%Y-%m-%d'))
df[cat_cols] = df[cat_cols].apply(pd.to_numeric, errors='coerce')

In [288]:
# remaining columns for OHE
cat_ohe_cols

['stat_flag',
 'cltsex_fix',
 'race_desc',
 'hh_size_est',
 'ctrycode_desc',
 'clttype',
 'annual_income_est']

# 3. Remove Rows
- Unreasonable values:  
    1. n_months_last_bought_products contains -ve values

In [289]:
unreasonable_negative_values = df.n_months_last_bought_products < 0

In [290]:
df = df.loc[~unreasonable_negative_values].\
        reset_index(drop=True)
display(df)

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,flg_hlthclaim_839f8a_ever,recency_hlthclaim_839f8a,flg_hlthclaim_14cb37_ever,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,f_purchase_lh
0,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17979,3363260248,None,Singapore,P,LAPSED,2016-08-01,1994-05-24,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17980,c3509f0aef,Chinese,Singapore,P,ACTIVE,1990-03-20,1948-12-16,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17981,6e44cfb3e7,Chinese,Singapore,P,ACTIVE,2007-12-31,1967-12-06,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17982,dd4e362f9f,Others,Singapore,P,ACTIVE,2007-09-01,1986-04-01,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 4. Feature Engineering
1. Replace all countries except for "Singapore" as "Others" in `ctrycode_desc`
2. Derive `num_months_since_first_interaction` from `min_occ_date`
3. Derive `client_age` from `cltdob_fix`
4. Encode categorical columns with OHE
5. Aggregate ape, sumins, and prempaid columns to sum for overall of each   
   category

In [291]:
date_today = dt.datetime.now()
print(date_today)

2024-01-28 17:27:59.531481


In [292]:
# ctrycode_desc: group to SG and others
df.loc[df.ctrycode_desc != "Singapore", 'ctrycode_desc'] = "Others"

# getting number of months since first interaction
year_diff = date_today.year - df["min_occ_date"].dt.year # To-do
month_diff = date_today.month - df["min_occ_date"].dt.month
df["num_months_since_first_interaction"] = year_diff * 12 + month_diff

# getting clients' age
df["client_age"] = date_today.year - df["cltdob_fix"].dt.year

df.drop(
    ['min_occ_date','cltdob_fix'],
    axis = 1,
    inplace = True
)

In [293]:
# OHE
ohe_cols = ["clttype", "stat_flag", "race_desc"]
binary_cols = ["cltsex_fix", "ctrycode_desc"]
ordinal_cols = ["annual_income_est", "hh_size_est"]

In [294]:
unique_col_values = {}

print("OHE columns")
for col in ohe_cols:
    unique_values = df[col].unique().tolist()
    unique_values = [value for value in unique_values if value != None]
    unique_col_values[col] = unique_values
    print(col, unique_values)

print("\nBinary columns")
for col in binary_cols:
    unique_values = df[col].unique().tolist()
    unique_values = [value for value in unique_values if value != None]
    unique_col_values[col] = unique_values
    print(col, unique_values)

print("\nOrdinal categorical columns")
for col in ordinal_cols:
    unique_values = df[col].unique().tolist()
    unique_values = [value for value in unique_values if value != None]
    unique_col_values[col] = unique_values
    print(col, unique_values)

OHE columns
clttype ['P', 'G', 'C']
stat_flag ['ACTIVE', 'LAPSED', 'MATURED']
race_desc ['Chinese', 'Others', 'Indian', 'Malay']

Binary columns
cltsex_fix ['Female', 'Male']
ctrycode_desc ['Singapore', 'Others']

Ordinal categorical columns
annual_income_est ['C.60K-100K', 'D.30K-60K', 'A.ABOVE200K', 'B.100K-200K', 'E.BELOW30K']
hh_size_est ['1', '3', '4', '>4', '2', '0']


In [295]:
def map_annual_income_est(s):
    if s == None: 
        return None
    elif s.startswith('A'):
        return 1
    elif s.startswith('B'):
        return 2
    elif s.startswith('C'):
        return 3
    elif s.startswith('D'):
        return 4
    elif s.startswith('E'):
        return 5
    else:
        print(s)
        assert False # This LoC should never run
    
def map_hh_size_est(s):
    if s == None:
        return None
    elif s == '>4':
        return 5
    assert int(s) in list(range(5))
    return int(s)

In [296]:
for col in ohe_cols:
    unique_values = unique_col_values[col]
    for value in unique_values:
        new_col_name = f"{col}_{value}"
        df[new_col_name] = df.apply(lambda row: '' if row[col] is None else int(row[col] == value), axis=1)

In [297]:
for col in binary_cols:
    unique_values = unique_col_values[col]
    new_col_name = f"is_{unique_values[0]}"
    df[new_col_name] = df.apply(lambda row: '' if row[col] is None else int(row[col] == unique_values[0]), axis=1)

In [298]:
df["annual_income_est"] = df["annual_income_est"].map(map_annual_income_est)
df["hh_size_est"] = df["hh_size_est"].map(map_hh_size_est)

In [299]:
df = df.drop(columns=ohe_cols)
df = df.drop(columns=binary_cols)

In [300]:
ins_cols = df.filter(regex='^((ape_)|(sumins_)|(prempaid_))', axis=1).columns

In [301]:
# sum and whether individual has purchased ape_* related insurance: if purchased none 0 else 1
df["ape_sum"] = df.filter(regex='^(ape_)', axis=1).apply(lambda x:sum(x), axis=1)
df["ape_grp"] = df.filter(regex='^(ape_grp)', axis=1).any(axis=1).astype(int)
df["ape_ln"] = df.filter(regex='^(ape_ln)', axis=1).any(axis=1).astype(int)
df["ape_ltc"] = df.filter(regex='^(ape_ltc)', axis=1).any(axis=1).astype(int)
df["ape_inv"] = df.filter(regex='^(ape_inv)', axis=1).any(axis=1).astype(int)

# sum and whether individual has purchased sumins_* related insurance: if purchased none 0 else 1
df["sumins_sum"] = df.filter(regex='^(sumins_)', axis=1).apply(lambda x:sum(x), axis=1)
df["sumins_grp"] = df.filter(regex='^(sumins_grp)', axis=1).any(axis=1).astype(int)
df["sumins_ln"] = df.filter(regex='^(sumins_ln)', axis=1).any(axis=1).astype(int)
df["sumins_ltc"] = df.filter(regex='^(sumins_ltc)', axis=1).any(axis=1).astype(int)
df["sumins_inv"] = df.filter(regex='^(sumins_inv)', axis=1).any(axis=1).astype(int)

# sum and whether individual has purchased prempaid_* related insurance: if purchased none 0 else 1
df["prempaid_sum"] = df.filter(regex='^(prempaid_)', axis=1).apply(lambda x:sum(x), axis=1)
df["prempaid_grp"] = df.filter(regex='^(prempaid_grp)', axis=1).any(axis=1).astype(int)
df["prempaid_ln"] = df.filter(regex='^(prempaid_ln)', axis=1).any(axis=1).astype(int)
df["prempaid_ltc"] = df.filter(regex='^(prempaid_ltc)', axis=1).any(axis=1).astype(int)
df["prempaid_inv"] = df.filter(regex='^(prempaid_inv)', axis=1).any(axis=1).astype(int)

In [302]:
df.drop(columns=ins_cols,inplace=True)

In [303]:
df.to_csv('./data/data.csv')

# 5. Handling of Missing Data
    1. Impute columns with NA and 1 other unique value as boolean
    1. Remove columns with more than 80% missing entries

In [304]:
def check_na_cols(df):
    na_cols = []
    for z,x in zip(df.isna().sum().keys(),df.isna().sum()):
        if x>0:
            na_cols.append(z)
    return na_cols

In [305]:
# Columns that can be imputed as boolean
bool_imputed_cols = []
for c in df.columns:
    if df[c].isna().sum() > 0:
        if df[c].nunique() == 1:
            if 'n_months' not in c and 'lapse' not in c:
                bool_imputed_cols.append(c)
            print(c,df[c].unique())

is_dependent_in_at_least_1_policy [ 0. nan]
f_ever_declined_la [nan  1.]
lapse_ape_ltc_1280bf [ 0. nan]
lapse_ape_inv_dcd836 [ 0. nan]
lapse_ape_lh_d0adeb [ 0. nan]
lapse_ape_32c74c [ 0. nan]
n_months_since_lapse_ltc_1280bf [9999.   nan]
n_months_since_lapse_inv_dcd836 [9999.   nan]
n_months_since_lapse_lh_d0adeb [9999.   nan]
n_months_since_lapse_32c74c [9999.   nan]
flg_affconnect_show_interest_ever [nan  1.]
flg_affconnect_ready_to_buy_ever [nan  1.]
flg_hlthclaim_839f8a_ever [nan  1.]
flg_hlthclaim_14cb37_ever [nan  1.]
f_purchase_lh [nan  1.]


In [306]:
for c in bool_imputed_cols:
    valcounts = df[c].value_counts()
    if 1 not in valcounts.values:
        df[c] = df[c].apply(lambda x: 1 if pd.isna(x) else 0)
    else:
        df[c] = df[c].apply(lambda x: 0 if pd.isna(x) else 1)

In [307]:
# Column data indicates there are no customers who have lapsed these policies
n_months_cols = [
    'n_months_since_lapse_ltc_1280bf',
    'n_months_since_lapse_inv_dcd836',
    'n_months_since_lapse_lh_d0adeb',
    'n_months_since_lapse_32c74c'
    ]

df.drop(n_months_cols,axis=1,inplace=True)

# By extension, no amount incurred for the policies above
lapse_const_cols = [
    'lapse_ape_ltc_1280bf',
    'lapse_ape_inv_dcd836',
    'lapse_ape_lh_d0adeb',
    'lapse_ape_32c74c'
]
df.drop(lapse_const_cols,axis=1,inplace=True)


In [308]:
# For columns that are less than 20% NA values, impute with most frequent 
# value / median
df_len = len(df)
na_cols = check_na_cols(df)
for c in na_cols:
    if df[c].isna().sum()/df_len < 0.2:
        if df[c].dtype == 'int64':
            df[c].fillna(df[c].median(),inplace=True)
        elif df[c].dtype =='float64':
            df[c].fillna(df[c].median(),inplace=True)
        elif df[c].dtype =='object':
            df[c].fillna(df[c].value_counts().index[0])



In [314]:
# For lapse_ape columns, 0 indicates a
na_cols = check_na_cols(df)
for c in na_cols:
    if 'lapse_ape' in c:
        df[c].fillna(0,inplace=True)

In [315]:
na_cols = check_na_cols(df)
for x in na_cols:
    print(x,df[x].isna().sum())

flg_affconnect_lapse_ever 17174
n_months_since_visit_affcon 17174
recency_clmcon 17196
recency_clmcon_regis 17196
hlthclaim_amt 16535
recency_hlthclaim 16535
recency_hlthclaim_success 16707
recency_hlthclaim_unsuccess 17374
recency_hlthclaim_839f8a 17699
recency_hlthclaim_14cb37 16609
giclaim_amt 17539
recency_giclaim 17539


In [324]:
df['giclaim_amt'].unique()

array([         nan, 1.472500e+03, 8.147800e+02, 2.821000e+02,
       1.090936e+04, 1.735200e+02, 4.320600e+02, 3.506897e+04,
       0.000000e+00, 3.824600e+02, 1.333278e+04, 3.784300e+02,
       7.282200e+02, 1.225940e+03, 4.863290e+03, 3.681000e+01,
       4.909100e+02, 4.731000e+02, 7.303740e+03, 5.602000e+01,
       9.227000e+01, 1.000000e+02, 8.157300e+02, 4.612170e+03,
       1.670430e+03, 2.500000e+02, 7.500000e+02, 6.954300e+02,
       8.425000e+02, 2.375000e+02, 2.420000e+02, 8.509303e+04,
       2.200000e+03, 3.500000e+02, 2.000000e+01, 3.296700e+02,
       4.329100e+02, 1.000000e+03, 7.921900e+02, 3.822490e+03,
       1.241200e+02, 9.735000e+01, 1.200000e+02, 4.070000e+02,
       6.000000e+02, 3.516900e+02, 7.014510e+03, 2.000000e+02,
       1.210000e+02, 1.600000e+03, 4.000000e+02, 1.170000e+02,
       3.780272e+04, 1.357100e+03, 3.146928e+04, 2.878500e+02,
       9.480450e+03, 3.000000e+02, 3.825760e+03, 2.402190e+03,
       6.198980e+03, 5.410800e+02, 5.575000e+02, 4.7780

In [310]:
# for numeric columns with no zero, already, null would imply zero hence impute
# accordingly

for c in na_cols:
    if df[c].dtype in ['int64','float64'] and 0 not in df[c].value_counts():
        df[c].fillna(0,inplace=True)

In [311]:
na_cols = check_na_cols(df)
for c in na_cols:
    print(c,df[c].isna().sum())

lapse_ape_grp_6fc3e6 12589
lapse_ape_grp_de05ae 12589
lapse_ape_grp_945b5a 12589
lapse_ape_grp_6a5788 12589
lapse_ape_ltc_43b9d5 12589
lapse_ape_grp_9cdedf 12589
lapse_ape_grp_1581d7 12589
lapse_ape_grp_22decf 12589
lapse_ape_lh_507c37 12589
lapse_ape_lh_839f8a 12589
lapse_ape_inv_e9f316 12589
lapse_ape_grp_caa6ff 12589
lapse_ape_grp_fd3bfb 12589
lapse_ape_lh_e22a6a 12589
lapse_ape_grp_70e1dd 12589
lapse_ape_grp_e04c3a 12589
lapse_ape_grp_fe5fb8 12589
lapse_ape_grp_94baec 12589
lapse_ape_grp_e91421 12589
lapse_ape_lh_f852af 12589
lapse_ape_lh_947b15 12589
flg_affconnect_lapse_ever 17174
n_months_since_visit_affcon 17174
recency_clmcon 17196
recency_clmcon_regis 17196
hlthclaim_amt 16535
recency_hlthclaim 16535
recency_hlthclaim_success 16707
recency_hlthclaim_unsuccess 17374
recency_hlthclaim_839f8a 17699
recency_hlthclaim_14cb37 16609
giclaim_amt 17539
recency_giclaim 17539


In [312]:
df['prempaid_sum']

0         92114.0
1             0.0
2             0.0
3             0.0
4         36888.0
           ...   
17979         0.0
17980    393048.0
17981         0.0
17982      3816.0
17983         0.0
Name: prempaid_sum, Length: 17984, dtype: float64